In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing necessary library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
sns.distplot(df['Pawpularity'])

In [ ]:
import os 
import cv2

In [ ]:
# Set the path for loading image dataset.
os.chdir('../input/petfinder-pawpularity-score/train')

# We can find the size of each image data from this procedure
size_data = pd.DataFrame()
for file in os.listdir():
    imgg = cv2.imread(file)
    w,h,c = imgg.shape
    size_data=size_data.append([[w,h,c,imgg.size/3]])
size_data

In [ ]:
size_data[3].value_counts()

In [ ]:
train_img = []
for file in os.listdir():
    file = cv2.imread(file)
    file = cv2.resize(file , (64,64) , interpolation = cv2.INTER_AREA)
    train_img.append(file/255)
train_img[:1]

In [ ]:
train_img_name = []
for i in os.listdir():
    train_img_name.append(i)
train_img_name[:5]

In [ ]:
for name in train_img_name:
    if(name[:-4] == '.jpg'):
        print(name)


In [ ]:
import tensorflow as tf

In [ ]:
train_csv_data = pd.DataFrame()
for img ,name in zip(train_img , train_img_name):
    name = name[:-4]
    location = df[df['Id']== name].index[0]
    train_csv_data = train_csv_data.append([df.loc[location]])
train_csv_data

In [ ]:
train_csv_data = train_csv_data.reset_index().drop(['index'] , axis = 1)
train_csv_data

In [ ]:
image_1 = cv2.imread('./'+train_csv_data['Id'][0]+'.jpg')
plt.imshow(image_1)

In [ ]:
plt.imshow(train_img[0])

In [ ]:
os.chdir('../test')

for i in os.listdir():
    file = cv2.imread(i)
    print(file.shape)

In [ ]:
test_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file = cv2.resize(file , (64,64) , interpolation = cv2.INTER_AREA)
    test_img.append(file/255)
test_img[:1]

In [ ]:
test_img_name = []
for i in os.listdir():
    test_img_name.append(i)
test_img_name[:1]

In [ ]:
test_csv_data = pd.DataFrame()
for img,name in zip(test_img , test_img_name):
    name = name[:-4]
    location = test_csv[test_csv['Id'] == name].index[0]
    test_csv_data = test_csv_data.append([test_csv.loc[location]])
test_csv_data = test_csv_data.reset_index().drop(['index'] , axis= 1)
test_csv_data[:1]

In [ ]:
plt.imshow(test_img[0])

In [ ]:
train_csv_x = train_csv_data.drop(['Id','Pawpularity'],axis=1)
train_y = train_csv_data['Pawpularity']

test_csv_x = test_csv_data.drop(['Id'],axis=1)

In [ ]:
train_csv_x.shape

In [ ]:
csv_input = tf.keras.Input(shape = train_csv_x.shape[1:] , name = 'CSV_Input')
img_input = tf.keras.Input(shape = np.array(train_img).shape[1:] , name = 'IMG_input')
csv_hidden1 = tf.keras.layers.Dense(200, activation= 'elu' , kernel_initializer = 'he_normal' , name = 'CSV_Hidden1')(csv_input)
csv_hidden2 = tf.keras.layers.Dense(200, activation= 'elu' , kernel_initializer = 'he_normal' , name = 'CSV_Hidden2')(csv_hidden1)
csv_hidden3 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden3')(csv_hidden2)
csv_hidden4 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden4')(csv_hidden3)
csv_hidden5 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden5')(csv_hidden4)
csv_hidden6 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden6')(csv_hidden5)
csv_dropout = tf.keras.layers.Dropout(0.5, name ='CSV_Dropout')(csv_hidden6)

img_conv1 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv1')(img_input)
img_conv2 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv2')(img_conv1)
img_pooling1 = tf.keras.layers.MaxPooling2D(4, name= 'IMG_Max1')(img_conv2)

img_conv3 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv3')(img_pooling1)
img_conv4 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv4')(img_conv3)
img_pooling2 = tf.keras.layers.MaxPooling2D(4, name= 'IMG_Max2')(img_conv4)

img_conv5 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv5')(img_pooling2)
img_conv6 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv6')(img_conv5)
img_pooling3 = tf.keras.layers.MaxPooling2D(3, name= 'IMG_Max3')(img_conv6)

img_dropout = tf.keras.layers.Dropout(0.5, name = 'IMG_Dropout')(img_pooling3)
img_conv7 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv7')(img_dropout)

img_hidden1 = tf.keras.layers.Dense(300, activation='elu', kernel_initializer = 'he_normal',name='IMG_hidden1' ,use_bias=False)(img_conv7)
img_dropout1 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout1')(img_hidden1)

img_hidden2 = tf.keras.layers.Dense(300, activation='elu', kernel_initializer = 'he_normal',name='IMG_hidden2' ,use_bias=False)(img_dropout1)

img_gpool = tf.keras.layers.GlobalAvgPool2D(name = 'IMG_Gpool')(img_hidden2)

img_dropout2 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout2')(img_gpool)



csv_output = tf.keras.layers.Dense(1, name = 'CSV_Output')(csv_dropout)
img_output = tf.keras.layers.Dense(1,name = 'IMG_Output')(img_dropout2)

model = tf.keras.Model(inputs=[csv_input, img_input], outputs=[csv_output, img_output], name='MY_WORK')

In [ ]:
model.summary()

In [ ]:
# move you current directory to back.
os.chdir('../')
os.chdir('../')
os.chdir('../')
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
learning_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate = 0.002,
decay_steps = 10000,
decay_rate = 0.97)

opt = tf.keras.optimizers.Adam(learning_rate = learning_schedule)
model.compile(loss=['mse','mse'], loss_weights=[0.5, 0.5], optimizer = opt, metrics = tf.keras.metrics.RootMeanSquaredError())

epoch_number = 20

check_1 = tf.keras.callbacks.ModelCheckpoint('MY_WORK.h5', save_best_only=True, verbose=2)

In [ ]:
model.fit( 
    x= [train_csv_x, np.array(train_img)], y = [train_y, train_y], epochs=epoch_number, 
    validation_split=0.2, verbose =2, workers=3, batch_size = 100, validation_batch_size = 100,
    callbacks = [check_1])

In [ ]:
best_model = tf.keras.models.load_model('MY_WORK.h5')
csv_result, img_result = best_model.predict([test_csv_x, np.array(test_img)])
final_result = pd.DataFrame(0.5 * csv_result + 0.5 * img_result)
final_result.columns =['Pawpularity']
final_result

In [ ]:
for ids, paw in zip(test_csv_data['Id'], final_result['Pawpularity']):
    location = submission[submission['Id'] == ids].index[0]
    submission['Pawpularity'].loc[location] = paw
submission
submission.to_csv('./working/submission.csv',index=False)